# 💼 Strategy Tests: SMA, Risk-Adjusted & Optimizers
Assets: `^SP500TR`, `BTC-USD`
Market Proxy: `URTH` (iShares MSCI World ETF)
Risk-Free Rate: FRED TB3MS

In [17]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from data.load_raw import load_prices
from src.returns import Returns
from src.kpis import KPI
from features.riskfree_interest import RiskFreeRate
from src.strategies import Strategies


## Daten laden

In [18]:
# Preise für Assets und Markt laden
assets = ["^SP500TR", "BTC-USD"]
market_symbol = "URTH"  # Proxy für MSCI World
all_symbols = assets + [market_symbol]
start = "2015-01-01"
end = "2024-01-01"

df = load_prices(all_symbols, start, end)
price_assets = df[assets]
price_market = df[market_symbol]

[*********************100%***********************]  3 of 3 completed


## Renditen & Risk-Free Rate

In [19]:
# Log-Renditen
log_ret_assets = Returns.log(price_assets)
log_ret_market = Returns.log(price_market.to_frame()).iloc[:, 0]

# Risk-Free Rate (daily)
rf = RiskFreeRate.us_fred(log_ret_assets.index)

### 1. SMA Crossover

In [20]:
# 0/1 Signale
signals = Strategies.sma_crossover(price_assets, fast=50, slow=200)

# Portfolio-Return: Signal von gestern * Rendite heute
port_ret_sma = (signals.shift(1) * log_ret_assets).sum(axis=1)

# KPI-Report
report_sma = KPI.get_kpi_report(port_ret_sma, risk_free_rate=rf)
print(report_sma)
port_ret_sma.plot.show()

{'Ann. Return': 0.5719705784074521, 'Ann. Volatility': 0.5828848424274752, 'Sharpe Ratio': 0.9816123659521244, 'Sortino Ratio': 1.2632492730343903, 'Max Drawdown': -0.6949646546569002, 'Calmar Ratio': 0.8230211055695064, 'Value at Risk': 0.06266594311094532, 'Skewness': -0.0538685287564894, 'Kurtosis': 10.940643120522777}


AttributeError: 'PlotAccessor' object has no attribute 'show'

### 2. Risk-Adjusted Weighting

#### Methode: Sharpe

In [9]:
# Einmaliges Gewicht basierend auf sharpe
weights_sharpe = Strategies.risk_adjusted_weights(price_assets, method='sharpe', max_weight=0.8).iloc[0]
port_ret_sharpe = (weights_sharpe * log_ret_assets).sum(axis=1)
print("Gewichte (sharpe):", weights_sharpe.to_dict())
print("KPIs (sharpe):", KPI.get_kpi_report(port_ret_sharpe, risk_free_rate=rf))

Gewichte (sharpe): {'^SP500TR': 0.44179399783493567, 'BTC-USD': 0.5582060021650644}
KPIs (sharpe): {'Ann. Return': 0.35387598888445193, 'Ann. Volatility': 0.4197153330006202, 'Sharpe Ratio': 0.8429990593407642, 'Sortino Ratio': 1.0107702250455008, 'Max Drawdown': -0.630948267146122, 'Calmar Ratio': 0.5608637146831207, 'Value at Risk': 0.04489351013039915, 'Skewness': -0.9832067598627049, 'Kurtosis': 14.613710092006297}


#### Methode: Sortino

In [10]:
# Einmaliges Gewicht basierend auf sortino
weights_sortino = Strategies.risk_adjusted_weights(price_assets, method='sortino', max_weight=0.8).iloc[0]
port_ret_sortino = (weights_sortino * log_ret_assets).sum(axis=1)
print("Gewichte (sortino):", weights_sortino.to_dict())
print("KPIs (sortino):", KPI.get_kpi_report(port_ret_sortino, risk_free_rate=rf))

Gewichte (sortino): {'^SP500TR': 0.4333813720931633, 'BTC-USD': 0.5666186279068367}
KPIs (sortino): {'Ann. Return': 0.3575173334602379, 'Ann. Volatility': 0.4249403301153621, 'Sharpe Ratio': 0.8412027528409963, 'Sortino Ratio': 1.0084316729696536, 'Max Drawdown': -0.6363703060565351, 'Calmar Ratio': 0.5618070643108795, 'Value at Risk': 0.0454493534155286, 'Skewness': -0.9745230855958172, 'Kurtosis': 14.545653548531408}


#### Methode: Calmar

In [11]:
# Einmaliges Gewicht basierend auf calmar
weights_calmar = Strategies.risk_adjusted_weights(price_assets, method='calmar', max_weight=0.8).iloc[0]
port_ret_calmar = (weights_calmar * log_ret_assets).sum(axis=1)
print("Gewichte (calmar):", weights_calmar.to_dict())
print("KPIs (calmar):", KPI.get_kpi_report(port_ret_calmar, risk_free_rate=rf))

Gewichte (calmar): {'^SP500TR': 0.33602601364493584, 'BTC-USD': 0.6639739863550641}
KPIs (calmar): {'Ann. Return': 0.3996569004980168, 'Ann. Volatility': 0.4863600812059622, 'Sharpe Ratio': 0.8216145978369502, 'Sortino Ratio': 0.9911519719099853, 'Max Drawdown': -0.6936116970373538, 'Calmar Ratio': 0.5761968868245508, 'Value at Risk': 0.05198064524696914, 'Skewness': -0.8845416813403932, 'Kurtosis': 13.85499973748069}


### 3. Marktbezogene Optimierer

In [12]:
# Treynor-Optimierung
opt_tr = Strategies.optimize_treynor(price_assets, log_ret_market, rf)
print("Optimale Gewichte (Treynor):", opt_tr.iloc[0].to_dict())
port_ret_tr = (opt_tr.iloc[0] * log_ret_assets).sum(axis=1)
print(KPI.get_kpi_report(port_ret_tr, market_returns=log_ret_market, risk_free_rate=rf))

Optimale Gewichte (Treynor): {'^SP500TR': 0.0, 'BTC-USD': 1.0}
{'Ann. Return': 0.5451033448861652, 'Ann. Volatility': 0.7064086352102736, 'Sharpe Ratio': 0.7715746378863133, 'Sortino Ratio': 0.9445423057467562, 'Max Drawdown': -0.8303625055816857, 'Calmar Ratio': 0.656464304712686, 'Value at Risk': 0.07535837324124858, 'Skewness': -0.6844033729893472, 'Kurtosis': 12.368908974914822}


In [13]:
# Jensen-Optimierung
opt_jn = Strategies.optimize_jensen(price_assets, log_ret_market, rf)
print("Optimale Gewichte (Jensen):", opt_jn.iloc[0].to_dict())
port_ret_jn = (opt_jn.iloc[0] * log_ret_assets).sum(axis=1)
print(KPI.get_kpi_report(port_ret_jn, market_returns=log_ret_market, risk_free_rate=rf))

Optimale Gewichte (Jensen): {'^SP500TR': 0.0, 'BTC-USD': 1.0}
{'Ann. Return': 0.5451033448861652, 'Ann. Volatility': 0.7064086352102736, 'Sharpe Ratio': 0.7715746378863133, 'Sortino Ratio': 0.9445423057467562, 'Max Drawdown': -0.8303625055816857, 'Calmar Ratio': 0.656464304712686, 'Value at Risk': 0.07535837324124858, 'Skewness': -0.6844033729893472, 'Kurtosis': 12.368908974914822}


### 4. Marktunkorrellierte Optimierer

In [14]:
# Minimale Beta
opt_mb = Strategies.optimize_min_beta(price_assets, log_ret_market)
print("Optimale Gewichte (Min Beta):", opt_mb.iloc[0].to_dict())
port_ret_mb = (opt_mb.iloc[0] * log_ret_assets).sum(axis=1)
print(KPI.get_kpi_report(port_ret_mb, market_returns=log_ret_market, risk_free_rate=rf))

Optimale Gewichte (Min Beta): {'^SP500TR': 0.0, 'BTC-USD': 1.0}
{'Ann. Return': 0.5451033448861652, 'Ann. Volatility': 0.7064086352102736, 'Sharpe Ratio': 0.7715746378863133, 'Sortino Ratio': 0.9445423057467562, 'Max Drawdown': -0.8303625055816857, 'Calmar Ratio': 0.656464304712686, 'Value at Risk': 0.07535837324124858, 'Skewness': -0.6844033729893472, 'Kurtosis': 12.368908974914822}


In [15]:
# Minimale Korrelation
opt_mc = Strategies.optimize_min_corr(price_assets, log_ret_market)
print("Optimale Gewichte (Min Corr):", opt_mc.iloc[0].to_dict())
port_ret_mc = (opt_mc.iloc[0] * log_ret_assets).sum(axis=1)
print(KPI.get_kpi_report(port_ret_mc, market_returns=log_ret_market, risk_free_rate=rf))

Optimale Gewichte (Min Corr): {'^SP500TR': 0.0, 'BTC-USD': 1.0}
{'Ann. Return': 0.5451033448861652, 'Ann. Volatility': 0.7064086352102736, 'Sharpe Ratio': 0.7715746378863133, 'Sortino Ratio': 0.9445423057467562, 'Max Drawdown': -0.8303625055816857, 'Calmar Ratio': 0.656464304712686, 'Value at Risk': 0.07535837324124858, 'Skewness': -0.6844033729893472, 'Kurtosis': 12.368908974914822}


## Erklärung von `src/strategies.py`
- **sma_crossover:** 0/1-Signale pro Asset.
- **risk_adjusted_weights:** Globale Gewichtung via Sharpe, Sortino, Calmar.
- **optimize_treynor / optimize_jensen:** Maximierung marktbezogener KPIs.
- **optimize_min_beta / optimize_min_corr:** Minimierung von Beta/Korrelation.

Jede Optimierung iteriert Gewichte von 0–1 in 0.01-Schritten.